In [1]:
import requests
from bs4 import BeautifulSoup


In [2]:
# URL of the match report
url = 'https://www.transfermarkt.de/spielbericht/index/spielbericht/3812391'

# Define headers to make the request look more human
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'
}

# Send a GET request to the URL with the headers and store the response in a variable
response = requests.get(url, headers=headers)
response

<Response [200]>

In [3]:
# Parse the HTML content of the response using Beautiful Soup
soup = BeautifulSoup(response.content, 'html.parser')

In [4]:

# Find all <a> elements with class="sb-vereinslink" 
teams = soup.find_all('a', {'class': 'sb-vereinslink'})

teams_dict = {}

home_team_link = teams[0]['href']
home_team_name = teams[0].get_text()
home_team = {'team_link': home_team_link, 'team_name': home_team_name}

away_team_link = teams[0]['href']
away_team_name = teams[0].get_text()
away_team = {'team_link': away_team_link, 'team_name': away_team_name}

teams_dict = {'home_team': home_team, 'away_team': away_team}
teams_dict

{'home_team': {'team_link': '/fc-liverpool/startseite/verein/31/saison_id/2021',
  'team_name': 'FC Liverpool'},
 'away_team': {'team_link': '/fc-liverpool/startseite/verein/31/saison_id/2021',
  'team_name': 'FC Liverpool'}}

In [5]:
starting_lineups = soup.find_all('div', {'class': 'large-7 columns small-12 aufstellung-vereinsseite'})
home_starting_lineup = starting_lineups[0]
away_starting_lineup = starting_lineups[1]

home_players = []
for player in home_starting_lineup.find_all('div', {'class': 'aufstellung-spieler-container'}):
    profile_link = player.find('a')['href']
    name = player.find('a').get_text()
    top = player['style'].split(';')[0].split(':')[1].strip()
    left = player['style'].split(';')[1].split(':')[1].strip()
    home_players.append(({'name': name, 'profile_link': profile_link, 'top': top, 'left': left}))

teams_dict['home_team']['starting_lineup'] = home_players

away_players = []
for player in away_starting_lineup.find_all('div', {'class': 'aufstellung-spieler-container'}):
    profile_link = player.find('a')['href']
    name = player.find('a').get_text()
    top = player['style'].split(';')[0].split(':')[1].strip()
    left = player['style'].split(';')[1].split(':')[1].strip()
    away_players.append(({'name': name, 'profile_link': profile_link, 'top': top, 'left': left}))

teams_dict['away_team']['starting_lineup'] = away_players


In [9]:
def findNationalityFromProfile(profile_link):
    # URL of the match report
    url_base = 'https://www.transfermarkt.de'
    url = url_base + profile_link

    # Define headers to make the request look more human
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'
    }

    # Send a GET request to the URL with the headers and store the response in a variable
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    nationalitaet = soup.find('span', text='Nationalität:')

    if nationalitaet:
        nationality = nationalitaet.find_next_sibling('span').text.split()[0]
        print(nationality)
    else:
        nationality = "Unknown"
        print("ERROR: Nationality Unknown")
    return nationality

In [10]:
for player in teams_dict['home_team']['starting_lineup']:
    nationality = findNationalityFromProfile(player['profile_link'])
    player['nationality'] = nationality


Brasilien
Frankreich
Niederlande
Schottland
England
Brasilien
England
Spanien
Kolumbien
Senegal
Ägypten


In [12]:
for player in teams_dict['away_team']['starting_lineup']:
    nationality = findNationalityFromProfile(player['profile_link'])
    player['nationality'] = nationality

teams_dict['away_team']

Belgien
Österreich
Brasilien
Frankreich
Spanien
Brasilien
Deutschland
Kroatien
Uruguay
Brasilien
Frankreich


{'team_link': '/fc-liverpool/startseite/verein/31/saison_id/2021',
 'team_name': 'FC Liverpool',
 'starting_lineup': [OrderedDict([('name', 'Courtois'),
               ('profile_link', '/thibaut-courtois/profil/spieler/108390'),
               ('top', '80%'),
               ('left', '40%'),
               ('nationality', 'Belgien')]),
  OrderedDict([('name', 'Alaba'),
               ('profile_link', '/david-alaba/profil/spieler/59016'),
               ('top', '63%'),
               ('left', '28%'),
               ('nationality', 'Österreich')]),
  OrderedDict([('name', 'Militão'),
               ('profile_link', '/eder-militao/profil/spieler/401530'),
               ('top', '63%'),
               ('left', '52.5%'),
               ('nationality', 'Brasilien')]),
  OrderedDict([('name', 'Mendy'),
               ('profile_link', '/ferland-mendy/profil/spieler/291417'),
               ('top', '61%'),
               ('left', '7.5%'),
               ('nationality', 'Frankreich')]),
  Ordered

In [13]:
nationalities_away = [player['nationality'] for player in teams_dict['away_team']['starting_lineup']]
nationalities_away

['Belgien',
 'Österreich',
 'Brasilien',
 'Frankreich',
 'Spanien',
 'Brasilien',
 'Deutschland',
 'Kroatien',
 'Uruguay',
 'Brasilien',
 'Frankreich']

In [14]:
nationalities_home = [player['nationality'] for player in teams_dict['home_team']['starting_lineup']]
nationalities_home

['Brasilien',
 'Frankreich',
 'Niederlande',
 'Schottland',
 'England',
 'Brasilien',
 'England',
 'Spanien',
 'Kolumbien',
 'Senegal',
 'Ägypten']

In [15]:
top_home = [player['top'] for player in teams_dict['home_team']['starting_lineup']]
left_home = [player['left'] for player in teams_dict['home_team']['starting_lineup']]

In [16]:
left_home

['40%',
 '28%',
 '52.5%',
 '7.5%',
 '73%',
 '40%',
 '53%',
 '27%',
 '15%',
 '40%',
 '65%']

In [17]:
top_home

['80%', '63%', '63%', '61%', '61%', '39%', '28%', '28%', '10%', '3%', '10%']

In [30]:
top_away = [player['top'] for player in teams_dict['away_team']['starting_lineup']]
left_away = [player['left'] for player in teams_dict['away_team']['starting_lineup']]
pos_percentages = list(zip(top_away, left_away))
print(pos_percentages)

[('80%', '40%'), ('63%', '28%'), ('63%', '52.5%'), ('61%', '7.5%'), ('61%', '73%'), ('39%', '40%'), ('28%', '53%'), ('28%', '27%'), ('10%', '65%'), ('10%', '15%'), ('3%', '40%')]


In [31]:
scaled_list = []
for tup in pos_percentages:
    x = float(tup[0].replace("%", "")) / 100.0
    y = float(tup[1].replace("%", "")) / 100.0
    scaled_x = x / 0.8
    scaled_y = y / 0.8
    scaled_list.append((scaled_x, scaled_y))

In [32]:
scaled_list

[(1.0, 0.5),
 (0.7875, 0.35000000000000003),
 (0.7875, 0.65625),
 (0.7625, 0.09374999999999999),
 (0.7625, 0.9125),
 (0.4875, 0.5),
 (0.35000000000000003, 0.6625),
 (0.35000000000000003, 0.3375),
 (0.125, 0.8125),
 (0.125, 0.18749999999999997),
 (0.0375, 0.5)]

In [33]:
def map_clean_percentages_to_pixels(percentages):
    top_start, top_end = 1650, 2850
    left_start, left_end = 200, 1800
    mapped_positions = []
    for percentage in percentages:
        top = int(top_start + (top_end - top_start) * float(percentage[0]))
        left = int(left_start + (left_end - left_start) * float(percentage[1]))
        mapped_positions.append((left, top))
    return mapped_positions

last_test = map_clean_percentages_to_pixels(scaled_list)
last_test

[(1000, 2850),
 (760, 2595),
 (1250, 2595),
 (350, 2565),
 (1660, 2565),
 (1000, 2235),
 (1260, 2070),
 (740, 2070),
 (1500, 1800),
 (499, 1800),
 (1000, 1695)]

In [34]:
teams_dict['away_team']['starting_lineup']

[OrderedDict([('name', 'Courtois'),
              ('profile_link', '/thibaut-courtois/profil/spieler/108390'),
              ('top', '80%'),
              ('left', '40%'),
              ('nationality', 'Belgien')]),
 OrderedDict([('name', 'Alaba'),
              ('profile_link', '/david-alaba/profil/spieler/59016'),
              ('top', '63%'),
              ('left', '28%'),
              ('nationality', 'Österreich')]),
 OrderedDict([('name', 'Militão'),
              ('profile_link', '/eder-militao/profil/spieler/401530'),
              ('top', '63%'),
              ('left', '52.5%'),
              ('nationality', 'Brasilien')]),
 OrderedDict([('name', 'Mendy'),
              ('profile_link', '/ferland-mendy/profil/spieler/291417'),
              ('top', '61%'),
              ('left', '7.5%'),
              ('nationality', 'Frankreich')]),
 OrderedDict([('name', 'Carvajal'),
              ('profile_link', '/daniel-carvajal/profil/spieler/138927'),
              ('top', '61%'),
     

In [35]:
from PIL import Image

folder = "images/countries"
background = Image.open("images/background_2.png").convert("RGBA")

country_list = ["Frankreich", "Brasilien", "Nepal", "Frankreich", "Paraguay","Deutschland", "Österreich", "Brasilien", "Spanien", "Frankreich", "Brasilien"]
country_list = nationalities_home
foregrounds = []

for country in country_list:
    foreground = folder + '/' + country + ".png"
    foregrounds.append(foreground)


positions_4_3_3_off = [      (1000, 150), 
                        (750, 450), (1250, 450),
            (300, 550),                         (1700, 550),
                             (1000,850),
                (500,1000),                 (1500,1000),
            (300,1275),                         (1700,1275),
                             (1000,1350) 
                    ]


for i in range(len(foregrounds)):
    foreground = Image.open(foregrounds[i]).convert("RGBA")
    width, height = foreground.size
    positions = positions_4_3_3_off
    position = positions[i]
    position = (position[0] - width // 2, position[1] - height // 2)
    background.alpha_composite(foreground, position)


positions_away = last_test

country_list = ["Frankreich", "Brasilien", "Nepal", "Frankreich", "Paraguay","Deutschland", "Österreich", "Brasilien", "Spanien", "Frankreich", "Brasilien"]
country_list = nationalities_away
foregrounds = []

for country in country_list:
    foreground = folder + '/' + country + ".png"
    foregrounds.append(foreground)

for i in range(len(foregrounds)):
    foreground = Image.open(foregrounds[i]).convert("RGBA")
    width, height = foreground.size
    positions = positions_away
    position = positions[i]
    position = (position[0] - width // 2, position[1] - height // 2)
    background.alpha_composite(foreground, position)


background.save("output.png")